In [1]:
import os

In [2]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from airlinesSentiment.constants import *
from airlinesSentiment.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config




In [9]:
import os
import gdown
import zipfile
from airlinesSentiment import logger
from airlinesSentiment.utils.common import get_size


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        ''' 
        Fetch data from URL
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_files(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [13]:
try:
    config = ConfigurationManager()
    data_ingest_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingest_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e

[2025-06-17 16:17:45,891: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-17 16:17:45,961: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-17 16:17:45,972: INFO: common: created directory at: artifacts]
[2025-06-17 16:17:45,973: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-17 16:17:45,974: INFO: 3604886704: Downloading data from https://drive.google.com/file/d/1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89
To: /Users/satwik/Downloads/MLproj/airlines_sentiment_classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.13M/1.13M [00:00<00:00, 6.46MB/s]

[2025-06-17 16:17:49,089: INFO: 3604886704: Download data from https://drive.google.com/file/d/1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89/view?usp=sharing into file artifacts/data_ingestion/data.zip]
